In [15]:
import json, csv

Looking at the user request, they want to add a conversion rule to handle coordinate formatting where "36 ° 5 ′ 30 '' N " should become "36°5′30  N " (removing spaces around degree, minute, and second symbols).



In [30]:
def detokenize_text(text):
    """
    Fixes excessive tokenization in text by restoring normal spacing.

    Args:
        text (str): The tokenized text with spacing issues

    Returns:
        str: The detokenized text with normal spacing
    """
    # Replace literal \n with actual newlines
    text = text.replace('\\n', '\n')

    # Fix spaces before punctuation
    text = text.replace(' .', '.')
    text = text.replace(' ,', ',')
    text = text.replace(' :', ':')
    text = text.replace(' ;', ';')
    text = text.replace(' !', '!')
    text = text.replace(' ?', '?')
    text = text.replace(' )', ')')
    text = text.replace('( ', '(')

    # Fix spaces around hyphens and dashes
    text = text.replace(' - ', '-')
    text = text.replace(' -- ', ' -- ')  # Keep spaces around em-dashes

    # Fix possessives
    text = text.replace(" 's", "'s")
    text = text.replace(" n't", "n't")

    # Fix quotes - convert `` and '' to regular quotes
    # text = text.replace('``', '"')
    # text = text.replace("''", '"')
    # text = text.replace(' "', ' "')
    # text = text.replace('" ', '" ')

    import re
    text = text.replace(" ' ", "' ")
    text = re.sub(r"`` (.*?) ''", r'"\1"', text)
    # Fix numbers with colons (like ratios)
    text = re.sub(r'(\d+)\s+:\s+(\d+)\s+:\s+(\d+)', r'\1:\2:\3', text)

    # Fix coordinate formatting (degrees, minutes, seconds)
    text = re.sub(r'([\d\.]+)\s*°\s*([\d\.]+)\s*′\s*([\d\.]+)\s*\'\'\s*([NSEW])\s*', r'\1°\2′\3″\4 ', text)
    text = re.sub(r'([\d\.]+)\s*°\s*\s*([NSEW])\s*', r'\1°\2″ ', text)

    # Remove extra spaces
    text = re.sub(r' +', ' ', text)

    # Remove trailing/leading spaces on each line
    lines = [line.strip() for line in text.split('\n')]
    text = '\n'.join(lines)

    return text


# Example usage:
tokenized = "Sonoma diet : A diet based on portion control and centered around consuming `` power foods '' and Hans 's  door . 36 ° 5 ′ 30 '' N / 36.09167 ° N 115.17472 ° W "
detokenized = detokenize_text(tokenized)
print(detokenized)
# Output: Sonoma diet: A diet based on portion control and centered around consuming "power foods"


Sonoma diet: A diet based on portion control and centered around consuming "power foods" and Hans's door. 36°5′30″N / 36.09167°N″ 115.17472°W″


Looking at the user request, they want to:
1. Read a TSV file
2. Apply the `detokenize_text` function to columns 2 and 3 (indices 1 and 2)
3. Save the result as a JSON file using the headers from the TSV



The user wants to change the output format from JSON to JSONL (JSON Lines format), where each record is on a separate line without overall JSON array structure.



Looking at the user's request, they want to add tqdm progress bars to see the progress of the operations happening in the cell. I need to install tqdm first and then add progress bars to the apply operations on the DataFrame columns.



Looking at your request, you want to:
1. Read a JSONL file 
2. Apply `detokenize_text` to `input.text` and all `contexts[*].text` fields
3. Save the processed data as a JSON file



Looking at the user's request, they want to change the output format from JSON to JSONL (JSON Lines format), where each record is written as a separate JSON object on its own line.



Looking at the user request, they want to modify the cell to process all JSONL files in a directory instead of just a single file.



In [32]:
import json
import os
from tqdm import tqdm

# Directory containing JSONL files
directory_path = "../benchmark/nq_new/answerable_final/"

# Get all JSONL files in the directory
jsonl_files = [f for f in os.listdir(directory_path) if f.endswith('.jsonl') and not f.endswith('_processed.jsonl')]

print(f"Found {len(jsonl_files)} JSONL files to process: {jsonl_files}")

# Process each JSONL file
for jsonl_file in jsonl_files:
    jsonl_input_path = os.path.join(directory_path, jsonl_file)
    print(f"\nProcessing {jsonl_file}...")

    # Read JSONL file
    data = []
    with open(jsonl_input_path, 'r', encoding='utf-8') as f:
        for line in tqdm(f, desc=f"Reading {jsonl_file}"):
            if line.strip():
                data.append(json.loads(line.strip()))

    print(f"Loaded {len(data)} records from {jsonl_file}")

    # Process each record
    for record in tqdm(data, desc=f"Processing {jsonl_file}"):
        # Process input.text if it exists
        if 'input' in record and 'text' in record['input']:
            record['input']['text'] = detokenize_text(str(record['input']['text']))

        # Process contexts[*].text if contexts exist
        if 'contexts' in record and isinstance(record['contexts'], list):
            for context in record['contexts']:
                if isinstance(context, dict) and 'text' in context:
                    context['text'] = detokenize_text(str(context['text']))

    # Save to JSONL file
    jsonl_output_path = jsonl_input_path.replace('.jsonl', '_processed.jsonl')
    with open(jsonl_output_path, 'w', encoding='utf-8') as f:
        for record in tqdm(data, desc=f"Writing {jsonl_file}"):
            f.write(json.dumps(record, ensure_ascii=False) + '\n')

    print(f"Successfully processed and saved to {os.path.basename(jsonl_output_path)}")
    print(f"Processed {len(data)} records")

print(f"\nCompleted processing all {len(jsonl_files)} JSONL files!")


Found 52 JSONL files to process: ['nq-train-37.jsonl', 'nq-train-15.jsonl', 'nq-train-07.jsonl', 'nq-dev-01.jsonl', 'nq-train-35.jsonl', 'nq-train-08.jsonl', 'nq-train-47.jsonl', 'nq-train-34.jsonl', 'nq-train-02.jsonl', 'nq-train-06.jsonl', 'nq-train-03.jsonl', 'nq-train-44.jsonl', 'nq-train-33.jsonl', 'nq-train-09.jsonl', 'nq-train-48.jsonl', 'nq-train-42.jsonl', 'nq-train-12.jsonl', 'nq-train-19.jsonl', 'nq-train-32.jsonl', 'nq-train-27.jsonl', 'nq-train-31.jsonl', 'nq-train-00.jsonl', 'nq-train-14.jsonl', 'nq-dev-00.jsonl', 'nq-train-01.jsonl', 'nq-train-17.jsonl', 'nq-train-30.jsonl', 'nq-train-04.jsonl', 'nq-train-40.jsonl', 'nq-train-26.jsonl', 'nq-train-28.jsonl', 'nq-train-11.jsonl', 'nq-train-39.jsonl', 'nq-train-25.jsonl', 'nq-train-41.jsonl', 'nq-train-49.jsonl', 'nq-train-16.jsonl', 'nq-train-29.jsonl', 'nq-train-45.jsonl', 'nq-train-23.jsonl', 'nq-train-18.jsonl', 'nq-train-10.jsonl', 'nq-train-24.jsonl', 'nq-train-38.jsonl', 'nq-train-22.jsonl', 'nq-train-46.jsonl', 'nq-

Reading nq-train-37.jsonl: 2150it [00:00, 126766.01it/s]


Loaded 2150 records from nq-train-37.jsonl


Writing nq-train-37.jsonl: 100%|██████████| 2150/2150 [00:00<00:00, 70995.87it/s]


Successfully processed and saved to nq-train-37_processed.jsonl
Processed 2150 records

Processing nq-train-15.jsonl...


...

Successfully processed and saved to nq-train-21_processed.jsonl
Processed 2129 records

Processing nq-train-20.jsonl...


Reading nq-train-20.jsonl: 2051it [00:00, 109232.77it/s]


Loaded 2051 records from nq-train-20.jsonl


Writing nq-train-20.jsonl: 100%|██████████| 2051/2051 [00:00<00:00, 72194.21it/s]

Successfully processed and saved to nq-train-20_processed.jsonl
Processed 2051 records

Completed processing all 52 JSONL files!


Looking at the user request, they want to:
1. Read a TSV file
2. Apply the `detokenize_text` function to columns 2 and 3 (indices 1 and 2)
3. Save as JSONL format using headers
4. Show progress with tqdm
5. Don't force ASCII in output



In [33]:
import pandas as pd
from tqdm import tqdm
import json

# Read TSV file
tsv_file_path = "../benchmark/clapnq/fixed_passages.tsv"
df = pd.read_csv(tsv_file_path, sep='\t')

print(f"Loaded {len(df)} records from TSV file")
print(f"Columns: {list(df.columns)}")

# Apply detokenize_text to second and third columns (indices 1 and 2)
tqdm.pandas(desc="Processing column 2")
df.iloc[:, 1] = df.iloc[:, 1].astype(str).progress_apply(detokenize_text)

tqdm.pandas(desc="Processing column 3")
df.iloc[:, 2] = df.iloc[:, 2].astype(str).progress_apply(detokenize_text)

# Convert to JSONL format
jsonl_output_path = tsv_file_path.replace('.tsv', '.jsonl')
with open(jsonl_output_path, 'w', encoding='utf-8') as f:
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Writing JSONL"):
        record = row.to_dict()
        f.write(json.dumps(record, ensure_ascii=False) + '\n')

print(f"Successfully saved {len(df)} records to {jsonl_output_path}")


Loaded 178890 records from TSV file
Columns: ['id', 'text', 'title', 'docid']


Writing JSONL: 100%|██████████| 178890/178890 [00:07<00:00, 25265.18it/s]

Successfully saved 178890 records to ../benchmark/clapnq/fixed_passages.jsonl
